In [75]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from keras import models, layers
import tensorflow.keras.backend as K

In [4]:
csv_file = pd.read_csv('../../files/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()

In [5]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../files/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../files/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [10]:
for x in tqdm(csv_file['fixed_inputs'].values.tolist()):
    if os.path.exists(x) == False:
        print(x)

100%|██████████| 192556/192556 [00:00<00:00, 609019.94it/s]


In [11]:
for x in tqdm(csv_file['mask_paths'].values.tolist()):
    if os.path.exists(x) == False:
        print(x)

100%|██████████| 192556/192556 [00:00<00:00, 601387.24it/s]


In [12]:
csv_file['fixed_inputs'].values.tolist()[0]

'../../files/train_v2/00003e153.jpg'

In [13]:
csv_file = csv_file[csv_file['fixed_inputs'] != '../../files/train_v2/6384c3e78.jpg']

In [14]:
allowed_gpus = [0]
gpus = tf.config.list_physical_devices("GPU")
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, "GPU")
for gpu in final_gpu_list:
    tf.config.experimental.set_memory_growth(gpu, True)

strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


2023-05-22 22:39:24.903228: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-22 22:39:25.006706: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-22 22:39:25.006856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-22 22:39:25.007961: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [18]:
load_dotenv('../config_files/dev.env')

True

In [19]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size = test_size, random_state=42)
    train, val = train_test_split(train, test_size = test_size, random_state=42)
    return train, val, test

In [22]:
# csv_file = pd.read_csv
train, val, test = split_datasets(csv_file)

In [23]:
type_of_label = []
for x in train['EncodedPixels'].values.tolist():
    if type(x[0]) == str:
        type_of_label.append(1)
    else:
        type_of_label.append(0)

In [24]:
def read_imgs(img, mask, shape):
    img = tf.io.read_file(img)
    mask = tf.io.read_file(mask)
    img = tf.image.decode_jpeg(img, channels=3)
    mask = tf.image.decode_jpeg(mask, channels = 1)
    img = tf.image.resize(img, size = shape)
    mask = tf.image.resize(mask, size = shape)
    img = img / 255
    mask = mask / 255
    return img, mask

In [27]:
def get_data(data, shape = (256, 256), shuffle = True, repeat = True, batch = True, batch_size = 32):
    imgs, masks = data['fixed_inputs'].values.tolist(), data['mask_paths'].values.tolist()
    shapes = [shape for x in range(len(imgs))]
    tensor = tf.data.Dataset.from_tensor_slices((imgs, masks, shapes))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(256 * REPLICAS)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    tensor = tensor.map(read_imgs)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [28]:
train_dataset = get_data(train)

In [89]:
def create_model(input_shape = (32, 32, 3)):
    input_img = layers.Input(input_shape, name = 'RGB Input')
    pp_in_layer = input_img
    pp_in_layer = layers.GaussianNoise(0.1)(pp_in_layer)

    # Downsampling starts
    # first block
    # Actual paper has starting number of filters as 64
    c1 = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(pp_in_layer)
    c1 = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    # second block
    c2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # third block
    c3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # fourth block
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # fifth block
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c5)

    # upsampling starts
    # first block
    u6 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c6)

    # second block
    u7 = layers.Conv2DTranspose(32, (2, 2), strides=(2,2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c7)

    # third block
    u8 = layers.Conv2DTranspose(16, (2, 2), strides=(2,2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c8)

    # fourth block
    u9 = layers.Conv2DTranspose(16, (2, 2), strides=(2,2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(c9)

    # final layer
    d = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
    d = layers.Cropping2D((8, 8))(d)
    d = layers.ZeroPadding2D((8, 8))(d)

    # d = layers.UpSampling2D(16)(d)
    model = models.Model(inputs = [input_img], outputs = [d])
    return model

def loss_fun(true, preds, smooth = 1e-6):
    true, preds = K.flatten(true), K.flatten(preds)
    BCE = keras.losses.binary_crossentropy(true, preds)
    intersection = K.sum(K.dot(true, preds))
    dice_loss = 1 - ((2 * (intersection)) / (K.sum(true) + K.sum(preds) + smooth))
    Dice_BCE = BCE + dice_loss
    return Dice_BCE

def IOU(true, preds):
    true, preds = K.flatten(true), K.flatten(preds)
    preds = tf.where(preds > 0.5, 1, 0)
    intersection = K.sum(true * preds)
    unioun = K.sum(true + preds) - intersection
    return intersection / unioun

def compile_model(model):
    optim = keras.optimizers.SGD()
    loss = loss_fun
    metrics = IOU
    model.compile(
        optimizer = optim,
        loss = keras.losses.binary_crossentropy,
        # metrics = metrics
    )
    return model

In [90]:
train_dataset = get_data(train, shape = (32, 32), batch_size=128)
val_dataset = get_data(val, shape = (32, 32), batch_size=128, shuffle=False, repeat=False)
with strategy.scope():
    model = compile_model(create_model())
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (128 * REPLICAS),
        epochs = 3,
        verbose = 1,
        validation_data = val_dataset,
    )

Epoch 1/3


2023-05-22 23:56:44.219296: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
      type: DT_STRING
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 188722
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\026TensorSliceDataset:113"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
      shape {
      }
      shape {
        dim {
          size: 2
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_i

 266/1474 [====>.........................] - ETA: 22s - loss: 0.1169

KeyboardInterrupt: 